In [32]:
import os

import numpy as np
import torch
import accelerate

from rd3d.datasets import build_dataloader
from rd3d.core import Config
from rd3d import PROJECT_ROOT
from rd3d.models.dense_heads.point_segment_head import PointSegmentor

# cfg_file = "configs/voxformer/voxformer_res_4x2_80e_kitti_3cls.py"
# ckpt_file = '/home/nrsl/workspace/temp/voxformer/output/kitti_3cls/voxformer_res_4x2_80e_kitti_3cls/vfe_res_seg/train/ckpt/checkpoint_epoch_80.pth'

cfg_file = "configs/voxformer/voxformer_4x2_80e_kitti_3cls.py"
ckpt_file = '/home/nrsl/workspace/temp/voxformer/output/kitti_3cls/voxformer_4x2_80e_kitti_3cls/seg/train/ckpt/checkpoint_epoch_80.pth'

os.chdir(PROJECT_ROOT)
acc = accelerate.Accelerator()
cfg = Config.fromfile_py(cfg_file)
dataloader = build_dataloader(cfg.DATASET, cfg.RUN, training=False)
batch_dict = dataloader.dataset.collate_batch([dataloader.dataset[30]])
dataloader.dataset.load_data_to_gpu(batch_dict)
from rd3d.models import build_detector
from rd3d.core.ckpt import load_from_file
from rd3d.utils import viz_utils
from matplotlib import pyplot as plt

cfg.MODEL.DENSE_HEAD.SAMPLE_TOPK = False
model = build_detector(cfg.MODEL, dataset=dataloader.dataset).cuda()

load_from_file(ckpt_file, model)
model.eval()
# batch_dict = model.vfe(batch_dict)
# viz_utils.viz_scene(batch_dict['voxel_features'][:, :3])
batch_dict = model.dense_head(model.backbone_3d(model.vfe(batch_dict)))
cls_scores = batch_dict['cls_logits'].sigmoid()
cls_scores, pred_labels = cls_scores.max(dim=-1)
print(torch.bincount(pred_labels))
pred_labels += 1
threshold = cls_scores.topk(256)[0][-1]
pred_labels[cls_scores < threshold] = 0
print(torch.bincount(pred_labels))
points = batch_dict['point_coords']
print(points.shape)

print(plt.cm.Set1(0))
print(plt.cm.Set1(1))
print(plt.cm.Set1(2))
print(plt.cm.Set1(3))

points_list = points.split(batch_dict['voxel_numbers'].tolist(), dim=0)
labels_list = pred_labels.split(batch_dict['voxel_numbers'].tolist(), dim=0)

this_points = points_list[0].detach().cpu().numpy()
this_pred_labels = labels_list[0].detach().cpu().numpy()
this_colors = plt.cm.Set1(this_pred_labels)[:, :3]
viz_utils.viz_scene((this_points, this_colors), batch_dict['gt_boxes_list'][0])


[2023-12-20 21:55:30,824 cfg INFO] import module at root: /home/nrsl/workspace/temp/voxformer
[2023-12-20 21:55:30,825 cfg INFO] import module as config: configs.voxformer.voxformer_res_4x2_80e_kitti_3cls
[2023-12-20 21:55:30,830 dataset INFO] Loading KITTI dataset
[2023-12-20 21:55:30,958 dataset INFO] Total samples for KITTI dataset: 3769
[2023-12-20 21:55:31,313 ckpt INFO] load checkpoint /home/nrsl/workspace/temp/voxformer/output/kitti_3cls/voxformer_res_4x2_80e_kitti_3cls/vfe_res_seg/train/ckpt/checkpoint_epoch_80.pth to cuda:0
[2023-12-20 21:55:31,314 ckpt INFO] checkpoint trained from version: 0.5.2+0000000
[2023-12-20 21:55:31,343 ckpt INFO] loaded params for model (275/275)


tensor([2638, 2613,  253], device='cuda:0')
tensor([5248,   99,   88,   69], device='cuda:0')
torch.Size([5504, 3])
(0.8941176470588236, 0.10196078431372549, 0.10980392156862745, 1.0)
(0.21568627450980393, 0.49411764705882355, 0.7215686274509804, 1.0)
(0.30196078431372547, 0.6862745098039216, 0.2901960784313726, 1.0)
(0.596078431372549, 0.3058823529411765, 0.6392156862745098, 1.0)
